In [1]:
import pandas as pd

In [3]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 37.5 MB/s eta 0:00:0000:0100:01


In [4]:
dfjan = pd.read_parquet("data/yellow_tripdata_2022-01.parquet")
dffeb = pd.read_parquet('data/yellow_tripdata_2022-02.parquet')

In [5]:
dfjan.shape

(2463931, 19)

In [11]:
dfjan['duration'] = (dfjan.tpep_dropoff_datetime - dfjan.tpep_pickup_datetime).dt.total_seconds()/60

In [12]:
dfjan.duration.head()

0    17.816667
1     8.400000
2     8.966667
3    10.033333
4    37.533333
Name: duration, dtype: float64

In [14]:
dfjan.duration.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

In [23]:
dfjansub = dfjan.loc[(dfjan.duration >=1) & (dfjan.duration <=60)]

len(dfjansub)/len(dfjan)

0.9827547930522406

In [24]:
from sklearn.feature_extraction import DictVectorizer

In [25]:
dv = DictVectorizer()

In [32]:
df_train = dfjansub[['PULocationID','DOLocationID']].astype(str).to_dict(orient = 'records')

In [33]:
X_train = dv.fit_transform(df_train)

In [30]:
y_train = dfjansub['duration'].values

In [34]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [35]:
from sklearn.linear_model import LinearRegression

In [36]:
lr = LinearRegression()

In [37]:
lr.fit(X_train, y_train)

LinearRegression()

In [41]:
from sklearn.metrics import mean_squared_error

In [42]:
y_pred_train = lr.predict(X_train)
mean_squared_error(y_train, y_pred_train, squared=False)

6.98619083458946

In [47]:
dffeb['duration'] = (dffeb.tpep_dropoff_datetime - dffeb.tpep_pickup_datetime).dt.total_seconds()/60
dffeb = dffeb.loc[(dffeb.duration >= 1) & (dffeb.duration <=60)]
df_val = dffeb[['PULocationID','DOLocationID']].astype(str).to_dict(orient = 'records')
X_val = dv.transform(df_val)
y_val = dffeb['duration'].values

In [48]:
y_pred_val = lr.predict(X_val)
mean_squared_error(y_val, y_pred_val, squared=False)

7.78640785205543